Задание:

**1 Часть** - Попробуйте улучшить результаты нейронной сети LSTM  из практического занятия поэкспериментировав с параметрами нейронной сети (Размер словаря, длина отзыва, размер слоя эмбеддинг)

**2 Часть** - Возьмите данные отзывов о фильмах сайта Кинопоиск ру из соревнования https://www.kaggle.com/mikhailklemin/kinopoisks-movies-reviews и попробуйте обучить рекурентную нейронную сеть, которая будет предсказывать sentiment по аналогии с практическим занятием по LSTM

## **1 Часть задания**

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import sequence
from numpy import array
import logging
logging.getLogger('tensorflow').disabled = True
import pandas as pd
import csv

### При словаре 10000, длине отзыва 250 и размере слоя эмбеддинга равном 8

Импорт всех необходимых библиотек и подготовка данных, исходя из копии блокнота практики. Поэтому комментрии я убрала из кода

In [ ]:
vocab_size = 10000

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.imdb.load_data(num_words=vocab_size)
class_names = ["Negative", "Positive"]

**Создать карту для преобразования набора данных IMDB в удобочитаемые обзоры**

In [ ]:
word_index = tf.keras.datasets.imdb.get_word_index()
word_index = {k:(v+3) for k,v in word_index.items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNKNOWN>"] = 2
word_index["<UNUSED>"] = 3
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

1654784/1641221 [==============================] - 0s 0us/step


**Data Insight**




In [ ]:
allreviews = np.concatenate((x_train, x_test), axis=0)
print("Maximum review length: {}".format(len(max((allreviews), key=len))))
print("Minimum review length: {}".format(len(min((allreviews), key=len))))
result = [len(x) for x in allreviews]
print("Mean review length: {}".format(np.mean(result)))
print("")
print("Machine readable Review")
print("  Review Text: " + str(x_train[60]))
print("  Review Sentiment: " + str(y_train[60]))
print("")
print("Human Readable Review")
print("  Review Text: " + decode_review(x_train[60]))
print("  Review Sentiment: " + class_names[y_train[60]])

Maximum review length: 2494
Minimum review length: 7
Mean review length: 234.75892

Machine readable Review
  Review Text: [1, 13, 219, 14, 33, 4, 2, 22, 1413, 12, 16, 373, 175, 2711, 1115, 1026, 430, 939, 16, 23, 2444, 25, 43, 697, 89, 12, 16, 170, 8, 130, 262, 19, 32, 4, 665, 7, 4, 2, 322, 5, 4, 1520, 7, 4, 86, 250, 10, 10, 4, 249, 173, 16, 4, 3891, 6, 19, 4, 167, 564, 5, 564, 1325, 36, 805, 8, 216, 638, 17, 2, 21, 25, 100, 376, 507, 4, 2110, 15, 79, 125, 23, 567, 13, 2134, 233, 36, 4852, 2, 5, 81, 1672, 10, 10, 92, 437, 129, 58, 13, 69, 8, 401, 61, 1432, 39, 1286, 46, 7, 12]
  Review Sentiment: 0

Human Readable Review
  Review Text: <START> i saw this at the <UNKNOWN> film festival it was awful every clichéd violent rich boy fantasy was on display you just knew how it was going to end especially with all the shots of the <UNKNOWN> wife and the rape of the first girl br br the worst part was the q a with the director writer and writer producer they tried to come across as <UNKNOWN> 

**Pre-processing Data**

In [ ]:
review_length = 250

x_train = sequence.pad_sequences(x_train, maxlen = review_length,padding='post')
x_test = sequence.pad_sequences(x_test, maxlen = review_length,padding='post')
print("Shape Training Review Data: " + str(x_train.shape))
print("Shape Training Class Data: " + str(y_train.shape))
print("Shape Test Review Data: " + str(x_test.shape))
print("Shape Test Class Data: " + str(y_test.shape))
print("")

Shape Training Review Data: (25000, 250)
Shape Training Class Data: (25000,)
Shape Test Review Data: (25000, 250)
Shape Test Class Data: (25000,)



**Create and build LSTM Recurrent Neural Network**

In [ ]:
tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(
        units=32 # 32 LSTM units in this layer
    ))

In [ ]:
model = tf.keras.models.Sequential()
size_embedding = 32
model.add(
    tf.keras.layers.Embedding(
        input_dim = vocab_size, # The size of our vocabulary
        output_dim = size_embedding, # Dimensions to which each words shall be mapped
        input_length = review_length # Length of input sequences
    )
)
model.add(
    tf.keras.layers.Dropout(
        rate=0.2 # Случайным образом отключается 25% нейронов
    )
)
model.add(
    tf.keras.layers.LSTM(
        units=32 # 32 LSTM units in this layer
    )
)
model.add(
    tf.keras.layers.Dropout(
        rate=0.2
    )
)
model.add(
    tf.keras.layers.Dense(
        units=1,
        activation='sigmoid'
    )
)
model.compile(
    loss=tf.keras.losses.binary_crossentropy, # loss function
    optimizer=tf.keras.optimizers.Adam(), # optimiser function
    metrics=['accuracy']) # reporting metric
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 250, 32)           320000    
                                                                 
 dropout_4 (Dropout)         (None, 250, 32)           0         
                                                                 
 lstm_3 (LSTM)               (None, 32)                8320      
                                                                 
 dropout_5 (Dropout)         (None, 32)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 328,353
Trainable params: 328,353
Non-trainable params: 0
_________________________________________________________________


**Train the LSTM**

In [ ]:
history = model.fit(
    x_train, y_train,
    batch_size=128,
    epochs=6,
    validation_split=0.2,
    verbose=1
)

Epoch 1/6
157/157 [==============================] - 4s 17ms/step - loss: 0.6946 - accuracy: 0.5362 - val_loss: 0.6906 - val_accuracy: 0.5208
Epoch 2/6
157/157 [==============================] - 2s 14ms/step - loss: 0.6747 - accuracy: 0.5739 - val_loss: 0.5486 - val_accuracy: 0.7766
Epoch 3/6
157/157 [==============================] - 2s 14ms/step - loss: 0.6170 - accuracy: 0.6806 - val_loss: 0.6752 - val_accuracy: 0.5402
Epoch 4/6
157/157 [==============================] - 2s 13ms/step - loss: 0.6609 - accuracy: 0.5638 - val_loss: 0.6594 - val_accuracy: 0.5724
Epoch 5/6
157/157 [==============================] - 2s 13ms/step - loss: 0.5197 - accuracy: 0.7369 - val_loss: 0.4618 - val_accuracy: 0.8258
Epoch 6/6
157/157 [==============================] - 2s 13ms/step - loss: 0.4040 - accuracy: 0.8555 - val_loss: 0.4834 - val_accuracy: 0.8086


**Evaluate model with test data and view results**

In [ ]:
loss, acc = model.evaluate(x_test,y_test)

782/782 [==============================] - 5s 6ms/step - loss: 0.4884 - accuracy: 0.8066


Результат на тестовой выборке `accuracy = 0.8321`

In [ ]:
with open('results.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    spamwriter.writerow([vocab_size, review_length, size_embedding, loss, acc])

### Пример кода поиска лучших параметров размера словаря, длины отзыва и размера слоя эмбеддинга

In [ ]:
vocab_size = 10000

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.imdb.load_data(num_words=vocab_size)
class_names = ["Negative", "Positive"]

Средний размер отзыва, на практике было показано, составляет ~230 символов, поэтому вариация была от 200 до 250 символов.

In [ ]:
review_length = 200

x_train = sequence.pad_sequences(x_train, maxlen = review_length,padding='post')
x_test = sequence.pad_sequences(x_test, maxlen = review_length,padding='post')
print("Shape Training Review Data: " + str(x_train.shape))
print("Shape Training Class Data: " + str(y_train.shape))
print("Shape Test Review Data: " + str(x_test.shape))
print("Shape Test Class Data: " + str(y_test.shape))

Shape Training Review Data: (25000, 200)
Shape Training Class Data: (25000,)
Shape Test Review Data: (25000, 200)
Shape Test Class Data: (25000,)


In [ ]:
tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(
        units=32 # 32 LSTM units in this layer
    ))

In [ ]:
model = tf.keras.models.Sequential()
size_embedding = 64
model.add(
    tf.keras.layers.Embedding(
        input_dim = vocab_size,
        output_dim = size_embedding,
        input_length = review_length
)
model.add(
    tf.keras.layers.Dropout(
        rate=0.2
    )
)
model.add(
    tf.keras.layers.LSTM(
        units=32
    )
)
model.add(
    tf.keras.layers.Dropout(
        rate=0.2
    )
)
model.add(
    tf.keras.layers.Dense(
        units=1,
        activation='sigmoid'
    )
)
model.compile(
    loss=tf.keras.losses.binary_crossentropy,
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy'])
model.summary()

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_22 (Embedding)    (None, 200, 64)           640000    
                                                                 
 dropout_44 (Dropout)        (None, 200, 64)           0         
                                                                 
 lstm_25 (LSTM)              (None, 32)                12416     
                                                                 
 dropout_45 (Dropout)        (None, 32)                0         
                                                                 
 dense_22 (Dense)            (None, 1)                 33        
                                                                 
Total params: 652,449
Trainable params: 652,449
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(
    x_train, y_train,
    batch_size=128,
    epochs=15,
    validation_split=0.2,
    verbose=1
)

Epoch 1/15
157/157 [==============================] - 4s 15ms/step - loss: 0.5817 - accuracy: 0.6611 - val_loss: 0.3621 - val_accuracy: 0.8524
Epoch 2/15
157/157 [==============================] - 2s 12ms/step - loss: 0.3083 - accuracy: 0.8826 - val_loss: 0.3362 - val_accuracy: 0.8656
Epoch 3/15
157/157 [==============================] - 2s 12ms/step - loss: 0.2261 - accuracy: 0.9188 - val_loss: 0.3177 - val_accuracy: 0.8562
Epoch 4/15
157/157 [==============================] - 2s 13ms/step - loss: 0.1846 - accuracy: 0.9384 - val_loss: 0.3168 - val_accuracy: 0.8730
Epoch 5/15
157/157 [==============================] - 2s 13ms/step - loss: 0.1484 - accuracy: 0.9543 - val_loss: 0.3954 - val_accuracy: 0.8598
Epoch 6/15
157/157 [==============================] - 2s 13ms/step - loss: 0.1186 - accuracy: 0.9641 - val_loss: 0.3740 - val_accuracy: 0.8712
Epoch 7/15
157/157 [==============================] - 2s 12ms/step - loss: 0.1089 - accuracy: 0.9698 - val_loss: 0.4068 - val_accuracy: 0.8678

In [ ]:
loss, acc = model.evaluate(x_test,y_test)

782/782 [==============================] - 4s 5ms/step - loss: 0.5566 - accuracy: 0.8509


Добавляем запись в файл, чтобы потом удобнее было отобразить результаты и не захламлять код

In [ ]:
with open('results.csv', 'a', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    spamwriter.writerow([vocab_size, review_length, size_embedding, loss, acc])
    csvfile.close()

### **Результат выполнения первой части задания**

Чтение файла, в котором записаны все результаты работы модели и её изменения

In [ ]:
results_table = pd.read_csv('results.csv', header=None, names=['Размер словаря', 'Длина отзыва', 'Размер слоя эмбединга', 'Loss', 'Accuracy'])

Для начала, я увеличила количество эпох до 6, т.к. 3 эпохи показали наименьший результат. Также 3 и 9 эксперименты проводились с увеличением units LSTM слоя в 2 раза (64), однако это не дало отличительных результатов.

В 10 эксперименте я вернула 3 эпохи вместо 6. Результат стал лучше на несколько тысячных.
Помимо тех результатов, что вошли в таблицу, было проведено ещё несколько с размеров слоя эмбединга 128 и разным количеством эпох. Однако результаты были в разы меньше, чем некоторые результаты из таблицы (которые более 0.8), поэтому я не стала вносить их в результирующую таблицу.

In [ ]:
results_table

,Размер словаря,Длина отзыва,Размер слоя эмбединга,Loss,Accuracy
0,10000,250,32,0.488362,0.80660
1,10500,300,32,0.550098,0.75128
2,20000,250,32,0.450912,0.82256
3,10000,200,64,0.357148,0.86336
4,20000,250,64,0.674415,0.54376
5,15000,250,64,0.654005,0.60092
6,10000,250,64,0.420365,0.84488
7,10000,230,64,0.596928,0.71352
8,10000,230,32,0.568950,0.73332
9,10000,200,64,0.381491,0.86064


Таким образом, у меня получилось поднять точность модели только до` 0.868` при размере словаря `10000`, длине отзыва `200` и размере слоя эмбединга `64`

In [ ]:
results_table.iloc[results_table['Accuracy'].idxmax()]

Размер словаря           10000.000000
Длина отзыва               200.000000
Размер слоя эмбединга       64.000000
Loss                         0.344968
Accuracy                     0.868360
Name: 10, dtype: float64

## **2 Часть задания**

Импорт всех необходимых библиотек

In [ ]:
import nltk
import re
nltk.download("stopwords")
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer

### **Загрузка датасета и его просмотр**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Датасет благодарственно заимствован из общего чата у добрейшего человека, который смог поделиться ей 🙏

In [ ]:
from pandas.io.parquet import read_parquet
df = pd.read_parquet('/content/drive/MyDrive/kinopoisk_dataset.parquet')

In [ ]:
df.head()

,review,sentiment
0,"Нередко, мы представляем себя на месте главных...",neg
1,"Редкий случай прекрасной идеи, требующей разви...",neg
2,"Все мы слышали про этот фильм, особенно после ...",neu
3,"Как говорят критики, общей чертой фильмов Финч...",neg
4,Я бы не хотел стричь всех скандинавов под одну...,pos


Пустых значений нет, значит нет отзыва без тектового комментария

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131669 entries, 0 to 131668
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   review     131669 non-null  object
 1   sentiment  131669 non-null  object
dtypes: object(2)
memory usage: 2.0+ MB


Можно увидеть количество позитивных и негативных отзывов в выборке. Числа отличаюются от датасета, предложенного на кагле, видимо, здесь были добавлены новые отзывы

In [ ]:
print('Кол-во негативных отзывов:  ', len(df[df['sentiment']=='neg']))
print('Кол-во позитивных отзывов:  ', len(df[df['sentiment']=='pos']))
print('Кол-во нейтральных отзывов: ', len(df[df['sentiment']=='neu']))

Кол-во негативных отзывов:   19827
Кол-во позитивных отзывов:   87138
Кол-во нейтральных отзывов:  24704


### **Подготовка выборки**

Для того, чтобы обучить модель, необходимо подготовить выборку. Выборка содержит отзыв и целевую переменную, также в виде текста. Поэтому необходимо преобразовать столбец "review" и столбец "sentiment". А именно: приведение к одному регистру, удаление любых знаков кроме буквенных, удаление стоп-слов для чистоты выборки, лемматизация и конечная токенизация

In [ ]:
df['review'] = df['review'].str.lower()#приведение к нижнему регистру

In [ ]:
df['review'] = df['review'].str.replace('[^\w\s]','')#удаляем все знаки

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [ ]:
df['review'] = df['review'].str.replace('\n', ' ')#заменяем символы \n

Дальше необходимо применить частотное разложение слов (токенизация), чтобы передать данные для обучения в модель. Использую для этого готовую библиотеку `nltk`. Также эта библиотека поможет привести слова к одинаковой корневой форме

In [ ]:
#токенизация
df['review'] = df['review'].apply(word_tokenize)

Так как широковстречающиеся слова в словаре - предлоги, их необходимо удалить

In [ ]:
russian_stopwords = stopwords.words("russian")

In [ ]:
#добавляю слова в словарь стоп-слов, которые также присутствуют в выборке и которые необходимо удалить
russian_stopwords.extend(['это', 'который', 'всё', 'которые', 'которая'])

In [ ]:
#удаляем стоп-слова
df['review'] = df['review'].apply(lambda x: [item for item in x if item not in russian_stopwords])
df.head(3)

,review,sentiment
0,"[нередко, представляем, месте, главных, героев...",neg
1,"[редкий, случай, прекрасной, идеи, требующей, ...",neg
2,"[слышали, фильм, особенно, грандиозного, успех...",neu


In [ ]:
lemmatize = nltk.WordNetLemmatizer()#объявление класса лемматизатора
#приведение слов к конкретным корневым формам
df['review'] = df['review'].apply(lambda x: [lemmatize.lemmatize(word) for word in x])

Дальше столбец необходимо токенизировать - перевести слова в числа. Использовала для этого библиотеку keras

In [ ]:
vocab_size = 10000 #размер словаря

In [ ]:
tokenizer = Tokenizer(num_words=vocab_size)#объявление класса токенизатора
tokenizer.fit_on_texts(df['review'])#создание словаря на основе отзывов выборки

In [ ]:
#перевод токенизированных слов в массив цифр
df['review'] = tokenizer.texts_to_sequences(np.array(df['review']))

Так как необходимо было произвести предсказание аналогичное предсказанию imdb, то строки, содержащие нейтральные отзывы, я удалила из выборки

In [ ]:
df = df.drop(np.where(df['sentiment'] == 'neu')[0])#удаление нейтральных отзывов

Дальше необходимо произвести преобразование столбца "sentiment" к виду 0 и 1. Я не использовала здесь метод one-hot-encoded, так как не совсем понимаю его использование при таком обучении

In [ ]:
df['sentiment'] = df['sentiment'].replace('neg', 0)
df['sentiment'] = df['sentiment'].replace('pos', 1)

In [ ]:
df.head()#вывод записей для проверки

,review,sentiment
0,"[5873, 377, 166, 25, 241, 377, 326, 3276, 1979...",0
1,"[3244, 956, 1792, 665, 997, 2517, 7681, 115, 9...",0
3,"[518, 1601, 1720, 37, 7127, 743, 148, 324, 100...",0
4,"[441, 388, 116, 273, 250, 322, 656, 1991, 3161...",1
5,"[40, 6599, 2939, 989, 2932, 5554, 4345, 6918, ...",1


In [ ]:
#разделяю всю выборку на две части - для обучения и тестирования в соотношении 33% к 67%
x_train, x_test, y_train, y_test = train_test_split(df['review'].values, df['sentiment'].values, test_size = 0.33, random_state = 42)

Дальнейшее преобразование происходит по аналогии с преобразованиями данных для imdb

In [ ]:
allreviews = np.concatenate((x_train, x_test), axis=0)

In [ ]:
#смотрим максимальную и минимальную длину отзывов, а также среднее
print("Максимальная длина отзыва: {}".format(len(max((allreviews), key=len))))
print("Минимальная длина отзыва:  {}".format(len(min((allreviews), key=len))))
result = [len(x) for x in allreviews]
print("Средняя длина отзыва:      {}".format(np.mean(result)))

Maximum review length: 875
Minimum review length: 3
Mean review length: 147.27356477226985


In [ ]:
#определяю усредненную длину отзыва
review_length = 100

In [ ]:
#приведение каждого отзыва к выбранной длине
x_train = sequence.pad_sequences(x_train, maxlen = review_length,padding='post')
x_test = sequence.pad_sequences(x_test, maxlen = review_length,padding='post')

In [ ]:
print("Размер массива отзывов для обучения: " + str(x_train.shape))
print("Размер массива Y для обучения:       " + str(y_train.shape))
print("Размер массива отзывов для теста:    " + str(x_test.shape))
print("Размер массива Y для теста:          " + str(y_test.shape))

Shape Training Review Data: (88218, 100)
Shape Training Class Data: (88218,)
Shape Test Review Data: (43451, 100)
Shape Test Class Data: (43451,)


### **Сборка, обучение и тестирование модели**

Данный блок полностью такой же, как блок с блокнота практики. Единственное - я уже настроила модель на наиболее лучший результат `accuracy` относительно тестовой выборки

In [ ]:
tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(
        units=32
    ))

In [ ]:
model = tf.keras.models.Sequential()

model.add(
    tf.keras.layers.Embedding(
        input_dim = vocab_size,
        output_dim = 64,
        input_length = review_length
    )
)

model.add(
    tf.keras.layers.Dropout(
        rate=0.2
    )
)

model.add(
    tf.keras.layers.LSTM(
        units=32
    )
)

model.add(
    tf.keras.layers.Dropout(
        rate=0.2
    )
)

model.add(
    tf.keras.layers.Dense(
        units=1,
        activation='sigmoid'
    )
)

model.compile(
    loss=tf.keras.losses.binary_crossentropy,
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy'])

model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 100, 64)           640000    
                                                                 
 dropout_4 (Dropout)         (None, 100, 64)           0         
                                                                 
 lstm_5 (LSTM)               (None, 32)                12416     
                                                                 
 dropout_5 (Dropout)         (None, 32)                0         
                                                                 
 dense_4 (Dense)             (None, 1)                 33        
                                                                 
Total params: 652,449
Trainable params: 652,449
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(
    x_train, y_train,
    batch_size=128,
    epochs=6,
    validation_split=0.2,
    verbose=1
)

Epoch 1/6
448/448 [==============================] - 37s 79ms/step - loss: 0.2370 - accuracy: 0.9086 - val_loss: 0.1278 - val_accuracy: 0.9554
Epoch 2/6
448/448 [==============================] - 35s 79ms/step - loss: 0.0943 - accuracy: 0.9699 - val_loss: 0.1178 - val_accuracy: 0.9608
Epoch 3/6
448/448 [==============================] - 35s 78ms/step - loss: 0.0649 - accuracy: 0.9808 - val_loss: 0.1201 - val_accuracy: 0.9618
Epoch 4/6
448/448 [==============================] - 35s 78ms/step - loss: 0.0476 - accuracy: 0.9868 - val_loss: 0.1377 - val_accuracy: 0.9633
Epoch 5/6
448/448 [==============================] - 35s 79ms/step - loss: 0.0416 - accuracy: 0.9889 - val_loss: 0.1461 - val_accuracy: 0.9600
Epoch 6/6
448/448 [==============================] - 35s 77ms/step - loss: 0.0340 - accuracy: 0.9913 - val_loss: 0.1477 - val_accuracy: 0.9630


В итоге на 6 эпохах тестовый результат `accuracy = 0.9650`

In [ ]:
loss, acc = model.evaluate(x_test,y_test)

1104/1104 [==============================] - 11s 10ms/step - loss: 0.1381 - accuracy: 0.9650


### **Проверка модели на собственных данных**



Было выбрано 3 отзыва: один из которых позитивный, другой - негативный, а третий - нейтральный. Данные отзывы прошли преобразования такие же, как основная выборка, перед тем, как были переданы в модель

In [ ]:
review_neg = "это был ужасный фильм в котором было слишком много крови и насилия"
review_pos = "это лучший фильм который я когда-либо видел он великолепен и фантастичен и мне он понравился"
review_neu = "непонятный фильм я не знаю плохой он или хороший"

In [ ]:
#токенизация
review_neg_arr = word_tokenize(review_neg)
review_pos_arr = word_tokenize(review_pos)
review_neu_arr = word_tokenize(review_neu)

In [ ]:
#удаление стоп-слов
review_neg_arr = [item for item in review_neg_arr if item not in russian_stopwords]
review_pos_arr = [item for item in review_pos_arr if item not in russian_stopwords]
review_neu_arr = [item for item in review_neu_arr if item not in russian_stopwords]

In [ ]:
#лемматизация
review_neg_arr = [lemmatize.lemmatize(word) for word in review_neg_arr]
review_pos_arr = [lemmatize.lemmatize(word) for word in review_pos_arr]
review_neu_arr = [lemmatize.lemmatize(word) for word in review_neu_arr]

In [ ]:
#приведение к цифровому формату
reviews= tokenizer.texts_to_sequences((review_neg_arr, review_pos_arr, review_neu_arr))

In [ ]:
#дополнение до недостающей длины отзыва
reviews = sequence.pad_sequences(reviews, maxlen = review_length,padding='post')

In [ ]:
reviews

array([[4005,    1,  176,   73,  744, 1651,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0],
       [ 596,    1,  459, 2339,  198,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    

In [ ]:
#предсказание модели
rawprediction = model.predict(reviews)

In [ ]:
rawprediction

array([[0.0204294],
       [0.9984918],
       [0.9972646]], dtype=float32)

После предсказания модели её показания необходимо округлить к 0 или 1

In [ ]:
predictions = []
for i in rawprediction:
  for j in i:
    predictions.append(int(round(j)))
print(predictions)

[0, 1, 1]


In [ ]:
print("Отзыв: " + review_neg)
print("Этот отзыв ", 'негативный' if predictions[0]==0 else 'позитивный')

Отзыв: это был ужасный фильм в котором было слишком много крови и насилия
Этот отзыв  негативный


In [ ]:
print("Отзыв: " + review_pos)
print("Этот отзыв ", 'негативный' if predictions[1]==0 else 'позитивный')

Отзыв: это лучший фильм который я когда-либо видел он великолепен и фантастичен и мне он понравился
Этот отзыв  позитивный


In [ ]:
print("Отзыв: " + review_neu)
print("Этот отзыв ", 'негативный' if predictions[2]==0 else 'позитивный')

Отзыв: непонятный фильм я не знаю плохой он или хороший
Этот отзыв  позитивный


Вывод: В целом модель явно отличила негативный и позитивный отзывы. Однако нейтральный отзыв, так как подобного при обучении не было, она отнесла к позитивному. Если бы обучение проходило на трех показателях, то она могла бы отличить и данный отзыв. Спасибо за проверку работы